## examine_functions.ipynb
### WESmith 05/09/20
- look at the individual node functions using truth tables
- update: testing gen_truth_funcs()

In [11]:
import phial.experiment as ex
import itertools as it
import pandas as pd

In [2]:
def test_func(fun, N):
    return pd.DataFrame([(k, (fun(k))) for k in it.product([0,1], repeat=N)])

In [3]:
dd = dict(A=2, B=3, C=2, D=1)  # example input dict: 1,2,3 input cases, and a duplicate 2-input case

In [4]:
funcs = ex.gen_truth_funcs(dd)

In [5]:
funcs

{'A': [<function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>],
 'B': [<function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(inputs)>,
  <function phial.experiment.AllPerms.__call__.<locals>.node(input

In [6]:
node = 'A'
for k in funcs[node]:
    print(test_func(k, dd[node]))

        0  1
0  (0, 0)  0
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  0
        0  1
0  (0, 0)  1
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  0
        0  1
0  (0, 0)  0
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  0
        0  1
0  (0, 0)  1
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  0
        0  1
0  (0, 0)  0
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  1
        0  1
0  (0, 0)  1
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  1
        0  1
0  (0, 0)  0
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  1
        0  1
0  (0, 0)  1
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  1


In [7]:
node = 'B'
for k in funcs[node]:
    print(test_func(k, dd[node]))

           0  1
0  (0, 0, 0)  0
1  (0, 0, 1)  0
2  (0, 1, 0)  0
3  (0, 1, 1)  0
4  (1, 0, 0)  0
5  (1, 0, 1)  0
6  (1, 1, 0)  0
7  (1, 1, 1)  0
           0  1
0  (0, 0, 0)  1
1  (0, 0, 1)  0
2  (0, 1, 0)  0
3  (0, 1, 1)  0
4  (1, 0, 0)  0
5  (1, 0, 1)  0
6  (1, 1, 0)  0
7  (1, 1, 1)  0
           0  1
0  (0, 0, 0)  0
1  (0, 0, 1)  1
2  (0, 1, 0)  1
3  (0, 1, 1)  0
4  (1, 0, 0)  1
5  (1, 0, 1)  0
6  (1, 1, 0)  0
7  (1, 1, 1)  0
           0  1
0  (0, 0, 0)  1
1  (0, 0, 1)  1
2  (0, 1, 0)  1
3  (0, 1, 1)  0
4  (1, 0, 0)  1
5  (1, 0, 1)  0
6  (1, 1, 0)  0
7  (1, 1, 1)  0
           0  1
0  (0, 0, 0)  0
1  (0, 0, 1)  0
2  (0, 1, 0)  0
3  (0, 1, 1)  1
4  (1, 0, 0)  0
5  (1, 0, 1)  1
6  (1, 1, 0)  1
7  (1, 1, 1)  0
           0  1
0  (0, 0, 0)  1
1  (0, 0, 1)  0
2  (0, 1, 0)  0
3  (0, 1, 1)  1
4  (1, 0, 0)  0
5  (1, 0, 1)  1
6  (1, 1, 0)  1
7  (1, 1, 1)  0
           0  1
0  (0, 0, 0)  0
1  (0, 0, 1)  1
2  (0, 1, 0)  1
3  (0, 1, 1)  1
4  (1, 0, 0)  1
5  (1, 0, 1)  1
6  (1, 1, 0)  1
7  (1, 1

In [8]:
node = 'C'
for k in funcs[node]:
    print(test_func(k, dd[node]))

        0  1
0  (0, 0)  0
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  0
        0  1
0  (0, 0)  1
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  0
        0  1
0  (0, 0)  0
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  0
        0  1
0  (0, 0)  1
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  0
        0  1
0  (0, 0)  0
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  1
        0  1
0  (0, 0)  1
1  (0, 1)  0
2  (1, 0)  0
3  (1, 1)  1
        0  1
0  (0, 0)  0
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  1
        0  1
0  (0, 0)  1
1  (0, 1)  1
2  (1, 0)  1
3  (1, 1)  1


In [9]:
node = 'D'
for k in funcs[node]:
    print(test_func(k, dd[node]))

      0  1
0  (0,)  0
1  (1,)  0
      0  1
0  (0,)  1
1  (1,)  0
      0  1
0  (0,)  0
1  (1,)  1
      0  1
0  (0,)  1
1  (1,)  1


In [10]:
# test when incorrect inputs: should get ValueError
node = 'A'
for k in funcs[node]:
    print(test_func(k, 3))

ValueError: this function group requires a tuple with 2 inputs: 3 supplied